In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
# Loading the data in pandas dataframe
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")
test = pd.read_csv("../input/titanic/test.csv")
train = pd.read_csv("../input/titanic/train.csv")
print(train.info())
print(train.head())

In [ ]:
#dropping unwanted columns
X_train = train.drop(columns = ['PassengerId','Survived','Ticket','Name','Cabin'])
X_test = test.drop(columns = ['PassengerId','Ticket','Name','Cabin'])
y_train = train.Survived
y_test = gender_submission.Survived
X_train.info()

In [ ]:
# filling the missing values
print(X_train.info())
X_train.Age=X_train.Age.fillna(X_train.Age.median())
X_train.Fare=X_train.Fare.fillna(X_train.Age.median())
X_train.Age = X_train.Age.astype(int)
X_train.Fare = X_train.Fare.astype(int)
X_test.Age=X_test.Age.fillna(X_test.Age.median())
X_test.Fare=X_test.Fare.fillna(X_test.Age.median())
X_test.Age = X_test.Age.astype(int)
X_test.Fare = X_test.Fare.astype(int)
print(X_train.info())

In [ ]:
categorical_feature_mask = X_train.dtypes==object
#filter categorical columns using mask and turn it into a list
categorical_cols = X_train.columns[categorical_feature_mask].tolist()

In [ ]:
train['LastName'] = train.Name.apply(lambda x:x.split(sep=',')[0])
train.info()

In [ ]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()
X_train.Sex = le.fit_transform(X_train.Sex)
X_test.Sex = le.fit_transform(X_test.Sex)
"""X_train.Cabin = X_train.Cabin.fillna(X_train['Cabin'].value_counts().idxmax())
X_train.Cabin = le.fit_transform(X_train.Cabin)
X_test.Cabin = X_test.Cabin.fillna(X_test['Cabin'].value_counts().idxmax())
X_test.Cabin = le.fit_transform(X_test.Cabin)"""
X_train.Embarked = X_train.Embarked.fillna(X_train['Embarked'].value_counts().idxmax())
X_train.Embarked = le.fit_transform(X_train.Embarked)
X_test.Embarked = X_test.Embarked.fillna(X_test['Embarked'].value_counts().idxmax())
X_test.Embarked = le.fit_transform(X_test.Embarked)
#X_train.Ticket = X_train.Ticket.apply(lambda x: 'Normal class' if str(x).isdigit() else x)
#print(X_train.Ticket)
#print(X_train.groupby('Ticket').count())

# Encoding the categorical values with one hot encoder


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
onehotencoder = OneHotEncoder(categorical_features = [0,3,6]) 
#columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [1,6])], remainder='passthrough')
#dataset = np.array(columnTransformer.fit_transform(X_train))
X_ohe_train = onehotencoder.fit_transform(X_train).toarray() 
X_ohe_test = onehotencoder.fit_transform(X_test).toarray() 

In [ ]:
#Feature scaling needs to be done for continous columns
from sklearn.preprocessing import MinMaxScaler
min_max=MinMaxScaler()
# To scale data 
X_train_minmax=min_max.fit_transform(X_ohe_train)
X_test_minmax=min_max.fit_transform(X_ohe_test)

In [ ]:
#percentage of people survived
print("percentage of people survived :",round(y_train.sum()/y_train.count(),2)*100)
groupby_sex_total = train.groupby('Sex').sum().Survived
groupby_sex_survived = train.groupby('Sex').count().Survived
#percentage of people survived grouped by sex
print("percentage of people survived grouped by sex : \n",str(round(groupby_sex_total/groupby_sex_survived*100,2)))

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
lr = LinearRegression()
#X_train = MinMaxScaler().fit_transform(X_train)
lr.fit(X_ohe_train,y_train)
y_pred = lr.predict(X_ohe_test)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",lr.score(X_ohe_test, y_test))

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_ohe_train,y_train)

y_pred=clf.predict(X_ohe_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
Accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:",Accuracy)
from sklearn.metrics import roc_auc_score
roc_score = roc_auc_score(y_test, y_pred)
print("ROC_AUC_SCORE : ",roc_score)

In [ ]:
final_results = pd.DataFrame(y_pred, columns = ['Survived'])
final_results['PassengerId'] = gender_submission.PassengerId
final_results=final_results.set_index('PassengerId')
final_results.to_csv('FinalResults-'+str(round(Accuracy,2))+'.csv')